<a href="https://colab.research.google.com/github/mtxslv/CaixeiroViajante/blob/master/simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For optimization problems, we need to delimitate a method, called simplex. I will write it down here, but it will be writed again in MatLab, Scilab, etc.

Our goal is to maximize (or minimize) a function z given by 
> $z = \sum_{j = 1}^{n}c_jx_j$

This function is called **objective function** and is limited by the following inequations:
> $\sum_{j = 1}^{n}a_{ij}x_j \leq b_i ,(i=1,...,m) \land x_j \geq 0, (j=1...n)$

First of all, we need to add one slack variable inside each inequality. It will change the system of inequations, leading to the next system:

>$\sum_{j = 1}^{n}a_{ij}x_j + x_{n+i} = b_i ,(i=1,...,m) \land x_{n+i} \geq 0, (j=1...n)$

After that, we put the right member of z to the left side and get:
> $z - \sum_{j = 1}^{n}c_jx_j = 0$

If we suppose there are $0z's$ in each of the constraints equations and $0x_{n+i}$, we can get the system:
> $z - \sum_{j = 1}^{n}c_jx_j + 0x_{n+i}= 0 \\
0z + \sum_{j = 1}^{n}a_{ij}x_j + x_{n+i} = b_i $

This last system can go into a matrix, and from there we delimitate our algorithm.


In this first moment, we will consider our algorithm **just solves maximization problems**.

In order to create this code, we need to deal with a lot of issues.
> Division by zero must [return a great value](https://docs.python.org/2/library/functions.html#max)

> While looking for the minimum limitant, we cannot return negative values. We nest [maximum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.maximum.html) and a [argmin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmin.html#numpy.argmin) to deal with this issue.

>

In [0]:
#important importation
import numpy as np

In [0]:
# we are going to deal with division by zero, sometimes. 
# In order to handle this issue, let's define our division function

def my_division(a,b): # a/b
  import numpy as np
  import sys
  c = np.ones(a.shape)
  #print(a)
  #print(b)
  for i in range(a.shape[0]):
    if (b[i] == 0):
      c[i] = sys.float_info.max
    else:
      c[i] = a[i]/b[i]
  return c

In [0]:
def simplex(tabela, num_var_basic):
  """
  input
    tabela: a matriz aumentada, na forma canônica, que contém a coluna da variável independente z.
            A última coluna é considerada a dos termos idependentes
    num_var_basic: a quantidade de variáveis básicas na matriz canônica aumentada.
  output
    o valor ótimo de z.
  """
  import numpy as np
  # vamos definir algumas variáveis
  contador = 0
  # matriz a ser iterada
  mat = tabela[:,1:]

  
  #vetor das variáveis básicas
  # Tenha cuidado! estamos lidando com a matriz, não com a tabela. 
  # Logo, estamos com uma coluna a menos. Além disso, as colunas começam
  # com 0
  basis = np.array( range(num_var_basic, mat.shape[1]-num_var_basic+1 ) )
  
  # inicialização do vetor dos termos limitantes
  q = mat[:,-1]

  while(np.amin(mat[0,:])<0):
    
    if(contador == 6):
      break
    
    i = np.argmin(mat[0,:]) # i é a coluna da variável que vai entrar na base 
    q = my_division(mat[1:,-1],mat[1:,i]) # definindo os termos limitantes
    l = np.argmin( np.maximum(np.zeros(q.shape),q) )# the looking for the minimum limitant can just return non negativa values
                                                    # l means "the basis in this line will be casted out"
    mat[l+1,:] = mat[l+1,:]/mat[l+1,i]  # garantir que o termo pivô é unitário
    # pivoting the column
    for it in range(mat.shape[0]):
      if(it!=l+1):
        mat[it,:] = mat[it,:] - (mat[it,i])*mat[l+1,:]  
    basis[l] = i # aqui atualizamos a nova base, esse deve ser  o último passo
     
    print(mat)
    print(np.maximum(np.zeros(q.shape),q))
    print("contador =", contador)
    
    contador = contador + 1
  return mat
     

In [0]:
tabela_teste = np.array([[1, -12, -15, 0, 0, 0, 0, 0],
                [0, 1, 0, 1, 0, 0, 0, 3],
                [0, 0, 1, 0, 1, 0, 0, 4],
                [0, 1, 1, 0, 0, 1, 0,6],
                [0, 1, 3, 0, 0, 0, 1, 13]])
n_var_basic_test = 2


In [112]:
resposta = simplex(tabela_teste,n_var_basic_test)

[[-12   0   0  15   0   0  60]
 [  1   0   1   0   0   0   3]
 [  0   1   0   1   0   0   4]
 [  1   0   0  -1   1   0   2]
 [  1   0   0  -3   0   1   1]]
[1.79769313e+308 4.00000000e+000 6.00000000e+000 4.33333333e+000]
contador = 0
[[  0   0   0 -21   0  12  72]
 [  0   0   1   3   0  -1   2]
 [  0   1   0   1   0   0   4]
 [  0   0   0   2   1  -1   1]
 [  1   0   0  -3   0   1   1]]
[3.00000000e+000 1.79769313e+308 2.00000000e+000 1.00000000e+000]
contador = 1
[[ 0  0  0  0  0 12 72]
 [ 0  0  1  0  0 -1  2]
 [ 0  1  0  0  0  0  4]
 [ 0  0  0  0  1 -1  1]
 [ 0  0  0  1  0  0  0]]
[0.66666667 4.         0.5        0.        ]
contador = 2


In [100]:
resposta

array([[ 0,  0,  0,  0,  0, 12, 72],
       [ 0,  0,  1,  0,  0, -1,  2],
       [ 0,  1,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  1, -1,  1],
       [ 0,  0,  0,  1,  0,  0,  0]])

In [101]:
mat = tabela_teste[:,1:]
mat

array([[ 0,  0,  0,  0,  0, 12, 72],
       [ 0,  0,  1,  0,  0, -1,  2],
       [ 0,  1,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  1, -1,  1],
       [ 0,  0,  0,  1,  0,  0,  0]])

In [102]:
basis = np.array(range(2, mat.shape[1] - 2+1 )) # basis = np.array( range(num_var, mat.shape[1]-num_var+1 ) )
basis # B maps the columns of the basis variables. The first element is two (2). It means that the variable
# of column 2 guides that line. 

array([2, 3, 4, 5])

In [103]:
i = np.argmin(mat[0,:])# i é a coluna da variável que vai entrar na base 
i

0

In [104]:
ans = my_division(mat[1:,-1],mat[1:,i])
print(ans)

[1.79769313e+308 1.79769313e+308 1.79769313e+308 1.79769313e+308]


In [105]:
l = np.argmin( np.maximum(np.zeros(ans.shape),ans) )# the looking for the minimum limitant can just return non negativa values
# l means "the basis in this line will be casted out"
l

0

In [106]:
mat[l+1,i] # mat[l+1,i], é l+1 pois consideramos a linha zero da 
           # tabela como a linha zero da matriz. Logo, as linhas estão sempre defasadas de 1 
print(mat)
mat[l+1,:] = mat[l+1,:]/mat[l+1,i]  # garantir que o termo pivô é unitário

[[ 0  0  0  0  0 12 72]
 [ 0  0  1  0  0 -1  2]
 [ 0  1  0  0  0  0  4]
 [ 0  0  0  0  1 -1  1]
 [ 0  0  0  1  0  0  0]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [107]:
mat[1,:] + 0.5*mat[2,:]

array([-9.22337204e+18, -9.22337204e+18, -9.22337204e+18, -9.22337204e+18,
       -9.22337204e+18, -9.22337204e+18, -9.22337204e+18])

In [108]:
for it in range(mat.shape[0]):
  if(it!=l+1):
    mat[it,:] = mat[it,:] - (mat[it,i])*mat[l+1,:]
mat

array([[                   0,                    0,                    0,
                           0,                    0,                   12,
                          72],
       [-9223372036854775808, -9223372036854775808, -9223372036854775808,
        -9223372036854775808, -9223372036854775808, -9223372036854775808,
        -9223372036854775808],
       [                   0,                    1,                    0,
                           0,                    0,                    0,
                           4],
       [                   0,                    0,                    0,
                           0,                    1,                   -1,
                           1],
       [                   0,                    0,                    0,
                           1,                    0,                    0,
                           0]])

In [109]:
basis[l] = i # aqui atualizamos a nova base, esse deve ser  o último passo
basis

array([0, 3, 4, 5])